In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

# Class: Nutritional Deficiency Analysis
class DeficiencyAnalysis:
    def __init__(self, crops_df, nutrients_df, age_nutrients_df):
        """
        Initializes the DeficiencyAnalysis class with the provided dataframes.
        
        Parameters:
        crops_df: DataFrame containing crop details for districts
        nutrients_df: DataFrame containing nutrient composition of crops
        age_nutrients_df: DataFrame containing recommended nutrient intake per age group
        """
        self.crops_df = crops_df
        self.nutrients_df = nutrients_df
        self.age_nutrients_df = age_nutrients_df

    def analyze_deficiencies(self, district_name):
        """
        Analyzes nutrient deficiencies for a specified district and returns results.

        Parameters:
        district_name: Name of the district to analyze

        Returns:
        deficiencies: A dictionary containing nutrient deficiencies for each age group
        """
        # Filter data for the specified district
        district_data = self.crops_df[self.crops_df['district_name'].str.lower() == district_name.lower()]
        if district_data.empty:
            print(f"No data found for district '{district_name}'.")
            return None

        # Define nutrient columns and map age nutrients
        nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 
                            'vitamin c', 'iron', 'calcium', 'potassium', 'magnesium']
        nutrient_mapping = {
            'carbohydrates': 'carbohydrates (g)',
            'protein': 'protein (g)',
            'fiber': 'fiber (g)',
            'fat': 'fat (g)',
            'vitamin a': 'vitamin a (mcg)',
            'vitamin c': 'vitamin c (mg)',
            'iron': 'iron (mg)',
            'calcium': 'calcium (mg)',
            'potassium': 'potassium (mg)',
            'magnesium': 'magnesium (mg)'
        }

        # Merge district data with nutrient composition
        district_nutrients = district_data.merge(self.nutrients_df, on='crop', how='inner')
        nutrient_totals = district_nutrients[nutrient_columns].sum()

        # Calculate deficiencies
        deficiencies = {}
        for age_group in self.age_nutrients_df['group']:
            # Extract RDI for the age group and align columns
            age_rdi = self.age_nutrients_df[self.age_nutrients_df['group'] == age_group][list(nutrient_mapping.values())]
            age_rdi.columns = nutrient_columns
            deficiency = age_rdi.values[0] - nutrient_totals.values
            deficiency_percent = (deficiency / age_rdi.values[0]) * 100
            deficiencies[age_group] = {col: (val, perc) for col, val, perc in zip(nutrient_columns, deficiency, deficiency_percent) if val > 0}

        # Generate visualizations
        self._plot_deficiency_charts(deficiencies)
        return deficiencies

    def _plot_deficiency_charts(self, deficiencies):
        """
        Generates bar charts and heatmap for visualizing nutrient deficiencies.

        Parameters:
        deficiencies: Dictionary of nutrient deficiencies
        """
        # Bar chart: Nutrient intake (actual vs required) per age group
        for age_group, nutrients in deficiencies.items():
            nutrient_names = list(nutrients.keys())
            actual = [val[0] for val in nutrients.values()]
            required = [val[0] + (val[0] / ((100 - val[1]) / 100)) for val in nutrients.values()]

            plt.figure(figsize=(12, 6))
            plt.bar(nutrient_names, required, label="Required Intake", alpha=0.7, color='lightblue')
            plt.bar(nutrient_names, actual, label="Actual Intake", alpha=0.7, color='orange')
            plt.title(f"Age Group: {age_group} - Nutrient Intake")
            plt.ylabel("Nutrient Intake (g/mg)")
            plt.xticks(rotation=45)
            plt.legend()
            plt.tight_layout()
            plt.show()

        # Heatmap: Nutrient deficiencies across age groups
        heatmap_data = pd.DataFrame(deficiencies).T.applymap(lambda x: x[1] if isinstance(x, tuple) else 0)
        plt.figure(figsize=(12, 8))
        sns.heatmap(heatmap_data, annot=True, cmap="coolwarm", fmt=".1f")
        plt.title("Nutrient Deficiencies Across Age Groups")
        plt.tight_layout()
        plt.show()

# Class: Nutritional Deficiency Forecast
class DeficiencyForecast:
    def __init__(self, crops_df):
        """
        Initializes the DeficiencyForecast class with the provided crops DataFrame.

        Parameters:
        crops_df: DataFrame containing crop details for districts
        """
        self.crops_df = crops_df

    def predict_deficiencies(self, deficiencies):
        """
        Predicts future deficiencies for the next 10 years based on growth rates.

        Parameters:
        deficiencies: Dictionary of current deficiencies by age group and nutrient

        Returns:
        forecast: A dictionary containing predicted deficiencies for each age group
        """
        forecast = {}
        growth_rate = 0.05  # Assume 5% annual growth

        for group, nutrients in deficiencies.items():
            forecast[group] = {}
            for nutrient, (amount, _) in nutrients.items():
                # Calculate future deficiency for 10 years
                future_values = [amount * ((1 + growth_rate) ** year) for year in range(1, 11)]
                forecast[group][nutrient] = future_values

        # Generate visualizations
        self._plot_forecast(forecast)
        return forecast

    def _plot_forecast(self, forecast):
        """
        Plots line graphs to show predicted deficiencies for the next 10 years.

        Parameters:
        forecast: Dictionary of predicted deficiencies for each age group
        """
        for group, nutrients in forecast.items():
            years = list(range(1, 11))  # Next 10 years
            plt.figure(figsize=(14, 8))  # Create a new plot for each age group

            for nutrient, values in nutrients.items():
                # Ensure `values` matches `years` in length
                plt.plot(years, values, marker='o', label=nutrient)

            plt.title(f"Predicted Deficiencies for Age Group: {group}", fontsize=16)
            plt.xlabel("Years", fontsize=12)
            plt.ylabel("Deficiency (g/mg)", fontsize=12)
            plt.legend(fontsize=10)
            plt.grid(True)
            plt.tight_layout()
            plt.show()
            
crops_df = pd.read_csv('final_karnataka_dataset.csv')
nutrients_df = pd.read_csv('nutrients.csv')
age_nutrients_df = pd.read_csv('age_wise_nutrients_consumtion_final.csv')

analysis = DeficiencyAnalysis(crops_df, nutrients_df, age_nutrients_df)
district_name = 'chitradurga'  # Example district
deficiencies = analysis.analyze_deficiencies(district_name)

forecast = DeficiencyForecast(crops_df)
predicted_deficiencies = forecast.predict_deficiencies(deficiencies)


KeyError: 'crop'